In [2]:
import logging
from sacred import Experiment
import numpy as np
import seml

import torch
from torch.utils.data import DataLoader

import sys
sys.path.append('/nfs/homedirs/bindera/reference_implementations/dgcnn/pytorch')
from model import DGCNN
from src.adversarial_attacks_on_feature_dependent_graphs.data import *
from src.adversarial_attacks_on_feature_dependent_graphs.attacks import PGD

import sklearn.metrics as metrics

from collections import namedtuple

/nfs/homedirs/bindera/miniconda3/envs/test_torch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/nfs/homedirs/bindera/reference_implementations/dgcnn/pytorch/pretrained/model.1024.t7' 
data_folder = '../../../shared/modelnet/modelnet40_normal_resampled'
cuda= True
#hyparams:
test_batch_size= 16
k= 20
dropout= 0.5
emb_dims= 1024

In [3]:
data_train, data_val, data_test = get_dataset(name = 'modelnet40', data_folder = data_folder)

test_loader = DataLoader(data_test,
                            batch_size=test_batch_size, shuffle=True, drop_last=False)

device = torch.device("cuda" if cuda else "cpu")

Args = namedtuple('Args', ['emb_dims', 'k', 'dropout'])
args = Args(emb_dims=emb_dims, k=k, dropout=dropout)

attack_args_l2 = {"epsilon": 5, "norm": "2"}

#Try to load models
model = DGCNN(args).to(device)
model = nn.DataParallel(model)
model.load_state_dict(torch.load(model_path))
model = model.eval()

The size of train data is 9843
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_train_1024pts_fps.dat...
The size of test data is 2468
Load processed data from ../../../shared/modelnet/modelnet40_normal_resampled/modelnet40_test_1024pts_fps.dat...


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:

test_acc = 0.0
count = 0.0
test_true = []
test_pred = []
for data, label in test_loader:

    data, label = data.to(device), label.to(device).squeeze()
    data = data.permute(0, 2, 1)
    batch_size = data.size()[0]

    data = data.requires_grad_()
    logits = model(data)

    # logits = model(x)

    x_pert_l2 = fast_gradient_attack(logits=logits, x=data, y=torch.tensor(label).long(), epsilon=attack_args_l2["epsilon"], norm=attack_args_l2["norm"],
                            loss_fn=torch.nn.functional.cross_entropy)
    
    # data.x = x_pert_l2
    logits = model(x_pert_l2)

    preds = logits.max(dim=1)[1]
    test_true.append(label.cpu().numpy())
    test_pred.append(preds.detach().cpu().numpy())
test_true = np.concatenate(test_true)
test_pred = np.concatenate(test_pred)
test_acc = metrics.accuracy_score(test_true, test_pred)

In [15]:
torch.manual_seed(42)
image = torch.randn(1,100)
layer = torch.nn.Linear(100,4)
y = torch.LongTensor([1])
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.functional.cross_entropy
logits = softmax(layer(image))

In [16]:
loss = loss_fn(logits, y)

In [13]:
next(iter(data_test))

(array([[ 0.582119  , -0.04899561,  0.03268051],
        [-0.7692958 , -0.04026098,  0.18489869],
        [-0.00802633, -0.07137198, -0.99741745],
        ...,
        [ 0.13096035,  0.27497098,  0.8954464 ],
        [-0.22372468, -0.06038009, -0.02359409],
        [-0.03982628, -0.04320524, -0.94765943]], dtype=float32),
 0)

In [26]:
import torch.nn.functional as F
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
loss = F.cross_entropy(input, target)

In [18]:
gradient = torch.autograd.grad(loss, input)[0]

In [30]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randint(5, (3,), dtype=torch.int64)
target.shape

torch.Size([3])

In [13]:
loss.backward()

In [42]:
torch.randint(5, (3,), dtype=torch.int64).shape

torch.Size([3])

In [53]:
T = 5
torch.manual_seed(42)
#image = torch.randn(1,5)
layer = torch.nn.Linear(5,5)
layer.eval()
# y = torch.ones(1,1)
image = torch.randn(3, 5, requires_grad=True)
y = torch.randint(5, (3,), dtype=torch.int64)
#loss = F.cross_entropy(input, target)

# y = torch.randint(5, (3,), dtype=torch.int64)
softmax = torch.nn.functional.softmax
loss_fn = torch.nn.functional.cross_entropy
epsilon = 5
with torch.autograd.set_detect_anomaly(True):
    for _ in range(T):
        # logits = softmax(layer(image))
        logits = softmax(image)
        print(logits.shape)
        # print(logits.shape)
        # print(image.shape)
        loss = F.cross_entropy(logits, y)
        
        # loss.backward()
        gradient = torch.autograd.grad(loss, image)[0]

        print(loss)
        # gradient = image.grad
        avoid_zero_div = torch.tensor(1e-12, requires_grad=True)
        
        dims = tuple(range(1, len(gradient.shape)))
        l2_norm_manual = torch.sqrt(torch.max(avoid_zero_div, torch.sum(gradient ** 2, dims, keepdim=True)))
        delta = gradient / l2_norm_manual
        image = image + epsilon * delta


   

torch.Size([3, 5])
tensor(1.4612, grad_fn=<NllLossBackward0>)
torch.Size([3, 5])
tensor(1.8196, grad_fn=<NllLossBackward0>)
torch.Size([3, 5])
tensor(1.8922, grad_fn=<NllLossBackward0>)
torch.Size([3, 5])
tensor(1.9048, grad_fn=<NllLossBackward0>)
torch.Size([3, 5])
tensor(1.9048, grad_fn=<NllLossBackward0>)


In [19]:
gradient

tensor([[ 0.0155,  0.0454, -0.1709,  0.0282,  0.0818],
        [-0.2890,  0.0136,  0.0699,  0.0664,  0.1391],
        [ 0.0535,  0.0671,  0.0897,  0.0348, -0.2451]])

In [14]:
input.grad

tensor([[ 0.0155,  0.0454, -0.1709,  0.0282,  0.0818],
        [-0.2890,  0.0136,  0.0699,  0.0664,  0.1391],
        [ 0.0535,  0.0671,  0.0897,  0.0348, -0.2451]])

In [28]:
input.shape

torch.Size([3, 5])

NameError: name 'dsedr' is not defined